# Statefulness

The first bots built within the course are stateless, meaning they have a 
consistent response to a specific kind of message. 

**Policies** - The relationship of a user input to an action.

Using memory (slot filling) & adding statefulness provides context to
the conversation. This can be useful if the user signals for assistance as 
they get stuck - you can be aware of where they were when you last
interacted with them.

<p style="color:red;">Warning: rasa_nlu dependency problems means this part
of the course is also non-reproducible.</p>

Below shows how to add statefulness to a bot, in the context of ordering a
meal from Mucky Bs...


In [1]:
# Commonplace to use integers to signify states
# Define the initial state
START = 0

# Define the choice / choosing state
CHOOSE_MEAL = 1

# Define the state for when the order has been placed
PLACED = 2

# Set out the policy rules. Use the format dict(tuple:tuple)
# The key tuple should be the current state and the intent detected
# The value tuple should be the next state - the state to be progressed to
# and the message to respond with...
policy = {
    (START, "order"): (CHOOSE_MEAL, "ok, Crabby Patty or Heart Stopper Deluxe?"),
    (START, "none"): (START, "My apologies, I didn't understand that..."),
    (CHOOSE_MEAL, "specify_food"): (PLACED, "perfect, the meal is on its way!"),
    (CHOOSE_MEAL, "none"): (CHOOSE_MEAL, "Umm... You can choose from Crabby Patties or Heart Stoppers, sir..."),
}

# Here define a list of inputs form the user 
inputs = [
    "I'd like to shave my beard",
    "oh right, let me get a meal then",
    "I own a German Shephard (the dog)",
    "make it a Crabby Patty then"
]

# Call send_message() for each message
state = START
#for message in inputs:    
#    state = send_message(policy, state, message)

# send_message was abstracted away in the course. It has a rasa dependency.

# in a future exercise, the policy above is adapted so the values contain
# a pending state, when you need to divert the user through an authentication
# pattern before rejoining the order flow.

The interesting behaviour in the above example - the user asks an irrelevant question at
the start and the bot in the START state responds with a generic response:  

> BOT: "My apologies, I didn't understand that..."

In the next interaction the user course corrects and makes a statement where the 
`specify_food` intent was found. Then when the user subsequently asks another irrelevant
question about dog ownership, the state of the bot (`CHOOSE_MEAL`) allows it to ask
a relevant clarification question:

> BOT: "Umm... You can choose from Crabby Patties or Heart Stoppers, sir..."

### Giving Help

It's likely to be a good idea to include a few policy rules for an intent
`ask_explanation`.

### Rejection

By looking for the user intent `deny`, you can ensure your bot doesn't keep
making the same suggestion when the user is rejecting it. The approach is 
to use lists for `previous_suggestions` and `excluded_options`.

```
if intent == "deny":
    excluded_options.extend(previous_suggestions)
```

Then use excluded options to filter out your response:

```
results = [
      r 
      for r in find_restaurants(params, excluded_options) 
      if r[0] not in excluded_options
  ]
```

## Pending States

The user experience can be improved and complexity of the bot can be
reduced by stashing actions in the scope where your respond function is
defined. When the user confirms the action, you can execute it. But if the
user does not confirm it, you wipe the pending action.

In [ ]:
# Define an updated policy
def policy(intent):
    # Return "action_pending" when the user affirms
    if intent == "affirm":
        return "action_pending", None
    # Return "OK" if the user intent is "deny"
    if intent == "deny":
        return "OK", None
    if intent == "order":
        return "We're out of Crabby Patties, a Heart Stopper instead?", "Great, that's ordered!"

#The order of the returned objects are action and pending action.